#### Friday, March 22, 2024

mamba activate t4nlpacv

Before running this, March 2024 usage for OpenAI was sitting at $0.06 ...

And at the endish, it was $0.67 ... ! 

#Fine-Tuning OpenAI models

Copyright 2024 Denis Rothman

[OpenAI fine-tuning documentation](https://beta.openai.com/docs/guides/fine-tuning/)

Check the cost of fine-tuning your dataset on OpenAI before running the notebook.

Run this notebook cell by cell to:

1.Preparing the dataset     
2.Fine-tunng a model  
3.Running the fine-tuned model
4.Managing fine-tuned jobs and models     
5.Before leaving


## Installing OpenAI
Restart the runtime if necessary after installing openai and run the cell again to make sur that "import openai" is executed.

In [ ]:
# try:
#   import openai
# except:
#   !pip install openai
#   import openai

## Your API Key

In [ ]:
#You can retrieve your API key from a file(1)
# or enter it manually(2)

#Comment this cell if you want to enter your key manually.
#(1)Retrieve the API Key from a file
#Store you key in a file and read it(you can type it directly in the notebook but it will be visible for somebody next to you)
# from google.colab import drive
# drive.mount('/content/drive')
# f = open("drive/MyDrive/files/api_key.txt", "r")
# API_KEY=f.readline()
# f.close()

In [12]:
#(2) Enter your manually by
# replacing API_KEY by your key.
#The OpenAI Key
# import os
# os.environ['OPENAI_API_KEY'] =API_KEY
# openai.api_key = os.getenv("OPENAI_API_KEY")

In [1]:
import os
import openai

OPENAI_API_KEY_ = os.environ['OPENAI_API_KEY_'] 
openai.api_key = OPENAI_API_KEY_

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY_

# 1.Preparing the dataset

## 1.1. Preparing the data in JSON

In [2]:
#From Gutenberg to JSON
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import requests
from bs4 import BeautifulSoup
import json
import re

# First, fetch the text of the book
# Option 1: from Project Gutenberg
#url = 'http://www.gutenberg.org/cache/epub/4280/pg4280.html'
#response = requests.get(url)
#soup = BeautifulSoup(response.content, 'html.parser')

[nltk_data] Downloading package punkt to /home/rob/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Option 2: from the GitHub repository:
!curl -L https://raw.githubusercontent.com/Denis2054/Transformers-for-NLP-and-Computer-Vision-3rd-Edition/master/Chapter08/gutenberg.org_cache_epub_4280_pg4280.html --output "gutenberg.org_cache_epub_4280_pg4280.html"

# 2.8s

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1295k  100 1295k    0     0   549k      0  0:00:02  0:00:02 --:--:--  549k


In [4]:
# Open and read the downloaded HTML file
with open("gutenberg.org_cache_epub_4280_pg4280.html", 'r', encoding='utf-8') as file:
    file_contents = file.read()

In [5]:
# Parse the file contents using BeautifulSoup
soup = BeautifulSoup(file_contents, 'html.parser')

# Get the text of the book and clean it up a bit
text = soup.get_text()
text = re.sub('\s+', ' ', text).strip()

# Split the text into sentences
sentences = sent_tokenize(text)

# Define the separator and ending
prompt_separator = " ->"
completion_ending = "\n"

In [6]:
# Now create the prompts and completions
data = []
for i in range(len(sentences) - 1):
    data.append({
        "prompt": sentences[i] + prompt_separator,
        "completion": " " + sentences[i + 1] + completion_ending
    })

In [7]:
# Write the prompts and completions to a file
with open('kant_prompts_and_completions.json', 'w') as f:
    for line in data:
        f.write(json.dumps(line) + '\n')

In [8]:
import pandas as pd

# Load the data
df = pd.read_json('kant_prompts_and_completions.json', lines=True)
df

prompt  \
0     The Project Gutenberg Etext of The Critique of...   
1     Be sure to check the copyright laws for your c...   
2     We encourage you to keep this file, exactly as...   
3                         Please do not remove this. ->   
4     This header should be the first thing seen whe...   
...                                                 ...   
6122  78-79. is their motto, under which they may le...   
6123  As regards those who wish to pursue a scientif...   
6124  When I mention, in relation to the former, the...   
6125          The critical path alone is still open. ->   
6126  If my reader has been kind and patient enough ...   

                                             completion  
0      Be sure to check the copyright laws for your ...  
1      We encourage you to keep this file, exactly a...  
2                          Please do not remove this.\n  
3      This header should be the first thing seen wh...  
4      Do not change or edit it without written perm...  
...                                                 ...  
6122   As regards those who wish to pursue a scienti...  
6123   When I mention, in relation to the former, th...  
6124           The critical path alone is still open.\n  
6125   If my reader has been kind and patient enough...  
6126   End of Project Gutenberg's The Critique of Pu...  

[6127 rows x 2 columns]

##  1.2. Converting the data to JSONL

Answer the questions as necessary for your project.

In [12]:
# The first time I ran this, I did not notice this was asking me for a Y/n response ... so it just chugged away on nothing ... 

# OK. So do NOT run it from here ... open up a terminal window, navigate to this folder, ensure the t4nlpacv environment is installed, 
# and then run this command:
# !openai tools fine_tunes.prepare_data -f "kant_prompts_and_completions.json"

# Running it from here I was unable to see where to type in Y ... 


Analyzing...

- Your JSON file appears to be in a JSONL format. Your file will be converted to JSONL format
- Your file contains 6127 prompt-completion pairs
- All prompts end with suffix ` ->`
- All completions end with suffix `\n`

Based on the analysis we will perform the following actions:
- [Necessary] Your format `JSON` will be converted to `JSONL`


Your data will be written to a new JSONL file. Proceed [Y/n]: ^C



In [13]:
import json

# Open the file and read the lines
with open('kant_prompts_and_completions_prepared.jsonl', 'r') as f:
    lines = f.readlines()


In [14]:
# Parse and print a few lines
for line in lines[199:300]:
    data = json.loads(line)
    print(json.dumps(data, indent=4))

{
    "prompt": "For he found that it was not sufficient to meditate on the figure, as it lay before his eyes, or the conception of it, as it existed in his mind, and thus endeavour to get at the knowledge of its properties, but that it was necessary to produce these properties, as it were, by a positive a priori construction; and that, in order to arrive with certainty at a priori cognition, he must not attribute to the object any other properties than those which necessarily followed from that which he had himself, in accordance with his conception, placed in the object. ->",
    "completion": " A much longer period elapsed before physics entered on the highway of science.\n"
}
{
    "prompt": "A much longer period elapsed before physics entered on the highway of science. ->",
    "completion": " For it is only about a century and a half since the wise Bacon gave a new direction to physical studies, or rather\u2014as others were already on the right track\u2014imparted fresh vigour t

creating the file on openai

In [15]:
from openai import OpenAI
client = OpenAI()

In [16]:
file_response=client.files.create(
  # file=open("/content/kant_prompts_and_completions_prepared.jsonl", "rb"),
  file=open("kant_prompts_and_completions_prepared.jsonl", "rb"),
  purpose='fine-tune'
)

# Print option for maintenance
#print(file_response)


# 24.7s

In [17]:
# Extract the training file ID
file_id = file_response.id
print(file_id)

file-XheWTawaIYJCtEzbfCzvFOXl


# 2.Fine-tuning a model

In [18]:
from openai import OpenAI
client = OpenAI()

In [19]:
job_response=client.fine_tuning.jobs.create(
  training_file=file_id,
  model="babbage-002"
)

In [20]:
job_id = job_response.id
print(job_id)

ftjob-ifR7nhlYfkW0eBo7jpHKh3wv


# 3. Checking the status of the fine-tuning job

In [21]:
# Check the status of the fine-tuning job
job_details = client.fine_tuning.jobs.retrieve(job_id)
print(job_details)

FineTuningJob(id='ftjob-ifR7nhlYfkW0eBo7jpHKh3wv', created_at=1711146271, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=12, learning_rate_multiplier=2), model='babbage-002', object='fine_tuning.job', organization_id='org-haJsqZZsFiYB0xf3TtlGyMPg', result_files=[], status='validating_files', trained_tokens=None, training_file='file-XheWTawaIYJCtEzbfCzvFOXl', validation_file=None, user_provided_suffix=None)


In [22]:
# There may be a time lapse:
# 1.between the moment you run the fine-tuning job and its completion
# 2.between its completion and the server updates
# Check your email if you have activated OpenAI notifications
status = job_details.status
print(f"Job status: {status}")

Job status: validating_files


Checking the list of the fine-tuning jobs and their status

In [23]:
# List 10 fine-tuning jobs
job_list=client.fine_tuning.jobs.list(limit=10)

In [24]:
import json
from pprint import pprint

# Get the raw JSON string from the SyncCursorPage object
json_string = job_list.json()

# Convert the JSON string into a Python object
data = json.loads(json_string)

# Extract the data array
jobs = data.get('data', [])

# Format the data into a list of dictionaries
formatted_data = [
    {
        'id': job.get('id'),
        'created_at': job.get('created_at'),
        'status': job.get('status'),
        'training_file': job.get('training_file'),
        'model': job.get('model'),
        'model_name':job.get('fine_tuned_model')
    }
    for job in jobs
]

# Print the formatted data
pprint(formatted_data)

[{'created_at': 1711146271,
  'id': 'ftjob-ifR7nhlYfkW0eBo7jpHKh3wv',
  'model': 'babbage-002',
  'model_name': None,
  'status': 'validating_files',
  'training_file': 'file-XheWTawaIYJCtEzbfCzvFOXl'}]


(At this point, navigate to the [Fine-tuning](https://platform.openai.com/finetune) page of OpenAI where you can see the status of your fine-tuning job ... Select the job to see details on the right ... This thing seemed to run forever!)

![Fine Tuning Start](OpenAI_Fine_Tuning_2024-03-22_18-34.png)

...So I let this thing run, watching if the status get's updated without a refresh ... it does ... but damn is it slow! ... So it eventually finished ... starting at 18:24 ... and starting fine tuning at 18:46 ... and finishing at  18:53:44 .... damn is that ever slooowwww!


![Fine Tuning Finish](OpenAI_Fine_Tuning_2024-03-22_18-55.png)


# 3.Running the fine-tuned model

We will now run the model for a completion task

Note: If your fine-tuned model does not appear immediately after the end of the fine-tuning process, you might have to wait until it is processed by OpenAI.

You can check the status regulary with the functions we just implemented above.

Check your email regularly for a confirmation also if you have activated OpenAI notifications.

In [28]:
# text to complete
text_content = "Space and time are key factors in human reasoning. Human minds cannot think without space and time perceptions."
#print(text_content)

In [29]:
prompt = "Continue the following text as if you were a scientist and philosopher" + text_content
#print(prompt)

In [32]:
response = client.completions.create(
    # model="ft:babbage-002:personal::8g9QuR5t",
    model="ft:babbage-002:personal::95i3ZApf",
    prompt=prompt,
    max_tokens=1000,
    temperature=0.8
)
#print(response)

In [33]:
# Check if there are any choices in the response
if response.choices:
    # Get the first choice (index 0)
    first_choice = response.choices[0]

    # Print the text of the first choice
    print("Model's Completion:", first_choice.text)
else:
    print("No choices returned in the response")

Model's Completion:  As regards phenomena and their causality, this is the condition of all intuition, whether given in experience or not.
As regards the objects of intuition, the sphere of phenomena is finite and wholly mental; the sphere of the possible world, infinite, and wholly necessary.
Text, 24.
(25)
ORIGIN AND CONSEQUENCES OF THE CONCEPT OF DUALITY.
I.
Thou art the object of my understanding, and I am the object of my faculty of understanding.
*II.
I perceive the relation of a thing in space and time to myself, and that to something else in the same time; and I perceive that, in the determination of this relation, there is something intrinsically different from the thing apprehended.
*III.
I connect the relation of this something with the possibility of the object of my understanding, and as the presupposition of its reality, and I perceive that, in the determination of this relation, the existence of something else is itself requisite.
*I do not say—as is often supposed—that,

In [34]:
# Formatting the response
print("Completion ID:", response.id)
print("Created:", response.created)
print("Model:", response.model)
print("Object Type:", response.object)

# Formatting the choices
for i, choice in enumerate(response.choices):
    print(f"Choice {i}:")
    print("  Finish Reason:", choice.finish_reason)
    print("  Index:", choice.index)
    print("  Logprobs:", choice.logprobs)
    print("  Text:", choice.text)

# Formatting the usage
print("Usage:")
print("  Completion Tokens:", response.usage.completion_tokens)
print("  Prompt Tokens:", response.usage.prompt_tokens)
print("  Total Tokens:", response.usage.total_tokens)

Completion ID: cmpl-95i8xmPPzrUGrlYl4Kx81aOUFXoAE
Created: 1711148355
Model: ft:babbage-002:personal::95i3ZApf
Object Type: text_completion
Choice 0:
  Finish Reason: length
  Index: 0
  Logprobs: None
  Text:  As regards phenomena and their causality, this is the condition of all intuition, whether given in experience or not.
As regards the objects of intuition, the sphere of phenomena is finite and wholly mental; the sphere of the possible world, infinite, and wholly necessary.
Text, 24.
(25)
ORIGIN AND CONSEQUENCES OF THE CONCEPT OF DUALITY.
I.
Thou art the object of my understanding, and I am the object of my faculty of understanding.
*II.
I perceive the relation of a thing in space and time to myself, and that to something else in the same time; and I perceive that, in the determination of this relation, there is something intrinsically different from the thing apprehended.
*III.
I connect the relation of this something with the possibility of the object of my understanding, and a

Once your fine-tune is available you can follow the following steps:

1.Go to the OpenAI Playground to test your model(the link is in the OpenAI email: https://platform.openai.com/playground

2.Then implement it in your environment

3.You can also run a classification fine-tunning example in this repository: https://github.com/Denis2054/Transformers-for-NLP-and-Computer-Vision-3rd-Edition/blob/main/Chapter08/Fine_tuned_classification.ipynb


# 4 Managing fine-tuned jobs and models

OpenAI offers several model management tools.

In [44]:
from openai import OpenAI
client = OpenAI()

# Set maintenance to True carefully if you wish to activate one of several
# job or model functions (information, cancel, delete)
maintenance=False
if maintenance is True:
  # List 10 fine-tuning jobs
  client.fine_tuning.jobs.list(limit=10)

  # Retrieve the state of a fine-tune
  client.fine_tuning.jobs.retrieve("ftjob-your job")

  # Cancel a job
  client.fine_tuning.jobs.cancel("ftjob-your job")

  # List up to 10 events from a fine-tuning job
  client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-your job", limit=10)

  # Delete a fine-tuned model (must be an owner of the org the model was created in)
  #client.models.delete("your model")

# 5.Before leaving

And what if a standard model can do the same job?

In [45]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
      "role": "user",
      "content": ""
    },
    {
      "role": "assistant",
      "content": "You are Kant, the philosopher"
    },
    {
      "role": "user",
      "content": "Explain why space and time are important from Kant's point of view"
    },
    {
      "role": "assistant",
      "content": ""
    }
  ],
  temperature=0.03,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [49]:
response.choices[0]

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='As Kant, I argue that space and time are not empirical concepts derived from external experiences, but rather they are fundamental structures of the mind, necessary for it to make sense of the world. This is a central part of my "transcendental aesthetic" theory.\n\nSpace and time are forms of intuition, which means they are the necessary conditions under which we can perceive objects. They are the "glasses" through which we view the world, so to speak. Without these glasses, we would not be able to perceive or understand anything.\n\nSpace is the form of outer intuition, meaning it is the framework within which we perceive physical objects as existing outside of us. Time, on the other hand, is the form of inner intuition, allowing us to perceive sequences of events and the passing of time.\n\nIn other words, space and time are not things that exist independently in the world, but rather they ar